In [21]:
from __future__ import division
from joblib import Parallel, delayed
from glob import glob
import numpy as np 
from imageio import imwrite, imread
import sys, getopt, os

from tile_utils import *

from scipy.stats import mode as md

In [30]:
# =========================================================
def writeout(tmp, cl, labels, outpath, thres):

   l, cnt = md(cl.flatten())
   l = np.squeeze(l)
   if cnt/len(cl.flatten()) > thres:
      outfile = id_generator()+'.jpg'
      fp = outpath+os.sep+labels[l]+os.sep+outfile
      imwrite(fp, tmp)

In [23]:
direc = 'train'
tile = 96
thres = 0.9

In [24]:
tile = int(tile)
thres = float(thres)

outpath = 'tile_'+str(tile)
files = sorted(glob(direc+os.sep+'*labels.npy'))

In [25]:
files

['train/usgs_pcmsc_2016_02_05_223925.TIF-0_ares_labels.npy',
 'train/usgs_pcmsc_2016_02_05_223945.TIF-0_ares_labels.npy']

In [26]:
labels = np.load(files[0])#['arr_0'].item(0)['labels']
#labels = [label.replace(' ','') for label in labels]

In [27]:
labels

array([u'anthro ', u'foam   ', u'terrain', u'water  ', u'veg    ',
       u'sand   ', u'road   '], dtype='<U7')

In [28]:
try:
   os.mkdir(outpath)
except:
   pass

for f in labels:
   try:
      os.mkdir(outpath+os.sep+f)
   except:
      pass

In [29]:
files = sorted(glob(direc+os.sep+'*class.npy'))

In [31]:
for f in files:
   print('working on '+str(f))
   res = np.load(f)
   fim = direc+os.sep+f.split(os.sep)[-1].replace('_ares_class.npy','.jpg')

   print('Generating tiles from dense class map ....')
   Z,ind = sliding_window(imread(fim), (tile,tile,3), (int(tile/2), int(tile/2),3)) 

   C,ind = sliding_window(res, (tile,tile), (int(tile/2), int(tile/2))) 

   w = Parallel(n_jobs=-1, verbose=0, pre_dispatch='2 * n_jobs', max_nbytes=None)(delayed(writeout)(Z[k], C[k], labels, outpath, thres) for k in range(len(Z))) 

working on train/usgs_pcmsc_2016_02_05_223925.TIF-0_ares_class.npy
Generating tiles from dense class map ....
working on train/usgs_pcmsc_2016_02_05_223945.TIF-0_ares_class.npy
Generating tiles from dense class map ....


'0.17.0'